In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 2.2.1     v purrr   0.2.4
v tibble  1.4.1     v dplyr   0.7.4
v tidyr   0.7.2     v stringr 1.2.0
v readr   1.1.1     v forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
#R <- as.matrix(read.csv("../input/resultstransitions/results-transitions.csv"))
R <- as.matrix(read.csv("./results-transitions.csv"))

#The configurations are 0 to 31, with 
#0 = 00000
#31 = 11111
#the proposed changes are
# 1 means a change in the 5th place = 16 (2**4)
# 2 means a change in the 4th place = 8  (2**3)
# 3 means a change in the 3rd place = 4  (2**2)
# 4 means a change in the 4th place = 2  (2**1)
# 5 means a change in the 1st place = 1  (2**0)
#because the changes are implemented from left to right on the binary number.
#so if the change is c then we need to use 2**(5-c)
#but the change only happens if the subsequent value is 1 corresponding to an acceptance of the change.

#1. Stereopsis (0 is mono, 1 is stereo)
#2. Parallax (0 is no parallax, 1 is parallax)
#3. Spatial sound (0 is mono, 1 is spatial)
#4. High resolution (0 is low, 1 is high)
#5. Colour (0 is original, 1 is alternative)

In [3]:
find_position_of_nine <- function(row) {
  if (length(row) < 2) {
    return(NA) # Return NA if the row has only one element
  }
  
  # Ignore the first element
  row <- row[-1]
  
  # Find the position of the first occurrence of 9
  position <- which(row == 9)[1]
  
  if (is.na(position)) {
    return(NA) # Return NA if 9 is not found
  }
  
  # Return the position - 1, as we want to exclude the 9 itself
  return((position - 1)/2)
}

In [4]:
#need to find the number of proposals for change
pos <- apply(R,1,find_position_of_nine)

In [5]:
setConfigurations <- function(){
   configs <- vector(mode="character")
   configs[1] <- "Monocular - No Parallax - Mono Audio - Low Resolution - Original Color"
   configs[2] <- "Monocular - No Parallax - Mono Audio - Low Resolution - Alternative Color"
   configs[3] <- "Monocular - No Parallax - Mono Audio - High Resolution - Original Color"
   configs[4] <- "configs[1] <- Monocular - No Parallax - Mono Audio - High Resolution - Alternative Color"
   configs[5] <- "Monocular - No Parallax - Spatial Audio - Low Resolution - Original Color"
   configs[6] <- "Monocular - No Parallax - Spatial Audio - Low Resolution - Alternative Color"
   configs[7] <- "Monocular - No Parallax - Spatial Audio - High Resolution - Original Color"
   configs[8] <- "Monocular - No Parallax - Spatial Audio - High Resolution - Alternative Color"
   configs[9] <- "Monocular - Parallax - Mono Audio - Low Resolution - Original Color"
   configs[10] <- "Monocular - Parallax - Mono Audio - Low Resolution - Alternative Color"
   configs[11] <- "Monocular - Parallax - Mono Audio - High Resolution - Original Color"
   configs[12] <- "Monocular - Parallax - Mono Audio - High Resolution - Alternative Color"
   configs[13] <- "Monocular - Parallax - Spatial Audio - Low Resolution - Original Color"
   configs[14] <- "Monocular - Parallax - Spatial Audio - Low Resolution - Alternative Color"
   configs[15] <- "Monocular - Parallax - Spatial Audio - High Resolution - Original Color"
   configs[16] <- "Monocular - Parallax - Spatial Audio - High Resolution - Alternative Color"
   configs[17] <- "Stereopsis - No Parallax - Mono Audio - Low Resolution - Original Color"
   configs[18] <- "Stereopsis - No Parallax - Mono Audio - Low Resolution - Alternative Color"
   configs[19] <- "Stereopsis - No Parallax - Mono Audio - High Resolution - Original Color"
   configs[20] <- "Stereopsis - No Parallax - Mono Audio High Resolution - Alternative Color"
   configs[21] <- "Stereopsis - No Parallax - Spatial Audio - Low Resolution Original Color"
   configs[22] <- "Stereopsis - No Parallax - Spatial Audio - Low Resolution - Alternative Color"
   configs[23] <- "Stereopsis - No Parallax - Spatial Audio - High Resolution - Original Color"
   configs[24] <- "Stereopsis - No Parallax - Spatial Audio - High Resolution - Alternative Color"
   configs[25] <- "Stereopsis - Parallax - Mono Audio - Low Resolution - Original Color"
   configs[26] <- "Stereopsis - Parallax - Mono Audio - Low Resolution - Alternative Color"
   configs[27] <- "Stereopsis - Parallax - Mono Audio - High Resolution - Original Color"
   configs[28] <- "Stereopsis - Parallax - Mono Audio - High Resolution - Alternative Color"
   configs[29] <- "Stereopsis - Parallax - Spatial Audio - Low Resolution - Original Color"
   configs[30] <- "Stereopsis - Parallax - Spatial Audio - Low Resolution - Alternative Color"
   configs[31] <- "Stereopsis - Parallax - Spatial Audio - High Resolution - Original Color"
   configs[32] <- "Stereopsis - Parallax - Spatial Audio  -  High Resolution - Alternative Color"
   
   return(configs)
}

In [6]:
configuration <- setConfigurations() 

factor_level <- matrix("",nrow=2,ncol=5) #the index into this is [change,accept+1] 
factor_level[1,1] <- "MonoVision"
factor_level[1,2] <- "NoParallax"
factor_level[1,3] <- "MonoSound"
factor_level[1,4] <- "LowRes"
factor_level[1,5] <- "OriginalColour"

factor_level[2,1] <- "StereoVision"
factor_level[2,2] <- "Parallax"
factor_level[2,3] <- "SpatialSound"
factor_level[2,4] <- "HighRes"
factor_level[2,5] <- "AltColour"

In [7]:
intToBinary <- function(n) {
  if (n == 0) return("0")
  
  bits <- rev(as.integer(intToBits(n)))
  start <- which(bits == 1)[1]
  paste0(bits[start:length(bits)], collapse = "")
}

In [8]:
N <- dim(R)
m <- N[1]
n <- N[2]
config <- matrix(-1,nrow = m, ncol = n)
changeAccept <- matrix(0,nrow=2,ncol=5) #first row means no change, second row change


In [9]:
#outcomes - the rows are vision, parallax, sound, resolution, colour
#the columns are 0->0 0->1 1->0 1->1
#a -> b meaning the configuration was in a but moved to b.
outcomes <- matrix(0,nrow=5,ncol=4)
#transition matrix
P <- matrix(0,nrow=32,ncol=32)

In [10]:
# n is the integer number
# position is the bit position you want to check (0-indexed, i.e., rightmost bit is 0)
check_bit <- function(n, position) {
  return(bitwAnd(n, 2**(5-position))>0)
}

In [11]:
#create a file for storing the individual outcomes
#change, enabled, accept
f <- file("result_binary.csv", "w")
writeLines("change,enabled,accept", f)
#change is the factor to be changed

In [12]:
for(i in 1:m){
  #get the initial configuration
  init <- R[i,1]
  config[i,1] <- R[i,1]
  for(j in seq(from=2,to=n,by=2)){
    #each subsequent pair are the proposed change and whether it is accepted or not
    #a 9 means that there are no subsequent changes so stop this row
    change <- R[i,j]
    if(change != 9){#there are more to come
      #is that feature already enabled?
      enabled <- check_bit(init,change)
      accept <- R[i,j+1] #accept==0 means no change, accept==1 means change
      if(!enabled & accept==0){#no change from 0 to 0:      0->0
        outcomes[change,1] <- outcomes[change,1] + 1
      }
      else if(!enabled & accept==1){# change from 0 to 1:   0->1
        outcomes[change,2] <- outcomes[change,2] + 1
      }
      else if(enabled & accept==0){#change from 1 to 0:     1->0
        outcomes[change,3] <- outcomes[change,3] + 1
      }
      else{#enabled & accept==1 change from 1 to 0:         1->1
        outcomes[change,4] <- outcomes[change,4] + 1
      }
      changeAccept[accept + 1,change] <- changeAccept[accept + 1,change] + 1
      newconfig <- bitwXor(init,(2**(5-change))*accept)
      P[init+1,newconfig+1] = P[init+1,newconfig+1] + 1 #transition
      init <- newconfig
      config[i,(j %/% 2)+1] <- init
      cat(cat(intToBinary(init)),' ')
      
      #write to the file
      line <- paste(change, as.integer(enabled), accept, sep = ",")
      writeLines(line, f)
    }
    else break
  }
  cat("\n")
}
close(f)


1010  1010  1110  1110  1110  1110  1110  11110  11110  11110  11110  11110  11110  11110  
1101  1100  100  101  1101  1101  1111  1110  1110  1111  1111  1110  1110  1111  1111  1110  
101  101  100  1100  1100  1110  1110  1110  1110  1110  110  110  110  110  110  
101  101  101  101  101  101  101  101  101  101  101  101  101  
11100  11101  11111  11111  11111  11111  11111  11111  11111  11111  
1111  1110  1110  1111  1111  1110  1110  1110  1110  1110  1110  1111  1111  1110  1110  1110  1110  1111  1111  1110  1010  1010  1110  
1110  1111  1111  1111  1111  1111  1111  1111  1111  1111  
10110  10110  10110  10110  10110  10110  10110  10110  10110  10110  
1101  1111  1111  1111  1111  111  110  110  110  110  110  10110  10110  10110  10110  10110  10110  10110  10110  10110  
10111  10110  10110  10110  10110  10110  10110  10110  10110  10110  
1001  1001  1101  1100  1100  1110  1110  1110  1110  110  111  111  111  111  111  111  10111  10111  10111  10111  10111  101

In [13]:
#note that 
colSums(changeAccept) #is the total number of changes suggested by factor irrespective of current state
rowSums(outcomes) #is the same, summing over all states


[1]  37  82  83  90 126

[1]  37  82  83  90 126

In [14]:
#outcomes is 0->0, 0->1, 1->0, 1->1 by factor 
#so outcomes[i,1] + outcomes[i,2] is the total number of times a change was proposed when factor i was off
# outcomes[i,3] + outcomes[i,4] is the total number when factor was on
propTimesChange <- matrix(0,nrow=5,ncol=2)
for(i in 1:5){
  s <- outcomes[i,1] + outcomes[i,2] #change proposed when i off
  propTimesChange[i,1] <- outcomes[i,2]/s
  s <- outcomes[i,3] + outcomes[i,4] #change proposed when i on
  propTimesChange[i,2] <- outcomes[i,4]/s
}
#propTimesChange[i,1] = proportion of times changed when i is off
#propTimesChange[i,2] = proportion of times changed when i is on
#For statistical analysis we need the binary variable 'accept' which
#is 0 or 1. For each trial we have the associated factor (condition) 
#1,2,...,5 and whether it was off (0) or on (1).
#In the above loop these are the variables
#change enabled accept and stored in the file result_binary.csv



In [15]:
result_binary <- read.csv("result_binary.csv")
rb <- result_binary
#note that sum(rb$accept[rb$change==5 & rb$enabled==1] == 1), must be equal to outcomes[5,4], etc.

In [16]:
acceptdata <- list(
                  n = length(rb$change),
                  monovision = as.integer(rb$change==1)*(1-rb$enabled),
                  stereovision = as.integer(rb$change==1)*rb$enabled,
                  noparallax = as.integer(rb$change==2)*(1-rb$enabled),
                  parallax = as.integer(rb$change==2)*rb$enabled,
                  monosound = as.integer(rb$change==3)*(1-rb$enabled),
                  spatialsound = as.integer(rb$change==3)*rb$enabled,
                  lowres = as.integer(rb$change==4)*(1-rb$enabled),
                  highres = as.integer(rb$change==4)*rb$enabled,
                  origcolour = as.integer(rb$change==5)*(1-rb$enabled),
                  newcolour = as.integer(rb$change==5)*rb$enabled,
                  accept = rb$accept
)


In [17]:
#Now the analysis with Stan
library("rstan")
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)


Loading required package: StanHeaders
rstan (Version 2.17.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Attaching package: 'rstan'

The following object is masked from 'package:tidyr':

    extract



In [18]:
stan_model <- "
// For the binary variable accept
data {
  int<lower=0> n; // number of observations
  int<lower=0, upper=1> accept[n]; // binary outcome
  int<lower=0, upper=1> monovision[n];
  int<lower=0, upper=1> stereovision[n];
  int<lower=0, upper=1> noparallax[n];
  int<lower=0, upper=1> parallax[n];
  int<lower=0, upper=1> monosound[n];
  int<lower=0, upper=1> spatialsound[n];
  int<lower=0, upper=1> lowres[n];
  int<lower=0, upper=1> highres[n];
  int<lower=0, upper=1> origcolour[n];
  int<lower=0, upper=1> newcolour[n];
}

transformed data{
  int k = 5; //number of parameters
}

parameters {
  real a0;
  real a1;
  vector[k] b0;
  vector[k] b1;
}

model {
  real m0;
  real m1;
  a0 ~ normal(0,10);
  a1 ~ normal(0,10);
  b0 ~ normal(0,10);
  b1 ~ normal(0,10);
  for(i in 1:n){
    /*
    accept[i] ~ bernoulli_logit(a + b[1]*monovision[i] + b[2]*stereovision[i]
                        + b[3]*noparallax[i] + b[4]*parallax[i] 
                        + b[5]*monosound[i] + b[6]*spatialsound[i] 
                        + b[7]*lowres[i] + b[8]*highres[i] 
                        + b[9]*origcolour[i]);
    */
    m0 = a0 + b0[1]*monovision[i] + b0[2]*noparallax[i] + b0[3]*monosound[i] +  b0[4]*lowres[i] + b0[5]*origcolour[i];
    m1 = a1 + b1[1]*stereovision[i] + b1[2]*parallax[i] + b1[3]*spatialsound[i] +  b1[4]*highres[i] + b1[5]*newcolour[i];
    accept[i] ~ bernoulli_logit(m0);
    accept[i] ~ bernoulli_logit(m1);
  }
}

generated quantities {
  vector[n] accept_new0;
  vector[n] accept_new1;
  vector[n] log_lik_accept0;
  vector[n] log_lik_accept1;
  real m0;
  real m1;

  for(i in 1:n){
    /*
    m = a + b[1]*monovision[i] + b[2]*stereovision[i]
          + b[3]*noparallax[i] + b[4]*parallax[i] 
          + b[5]*monosound[i] + b[6]*spatialsound[i] 
          + b[7]*lowres[i] + b[8]*highres[i] 
          + b[9]*origcolour[i];
    */
    m0 = a0 + b0[1]*monovision[i] + b0[2]*noparallax[i] + b0[3]*monosound[i] +  b0[4]*lowres[i] + b0[5]*origcolour[i];
    m1 = a1 + b1[1]*stereovision[i] + b1[2]*parallax[i] + b1[3]*spatialsound[i] +  b1[4]*highres[i] + b1[5]*newcolour[i];
    accept_new0[i] = bernoulli_logit_rng(m0);
    accept_new1[i] = bernoulli_logit_rng(m1);
    log_lik_accept0[i] = bernoulli_logit_lpmf(accept[i]|m0);
    log_lik_accept1[i] = bernoulli_logit_lpmf(accept[i]|m1);
  }
}
"

In [19]:
fit <- stan (model_code = stan_model, # Stan program
            data = acceptdata, # named list of data
            chains = 4, # number of Markov chains
            iter = 2000, # total number of iterations per chain
            cores = 4, 
            seed=54321)

Warning message:
No C++ compiler found, so the following will probably not work.rl-sm-vr/lib/R/bin//R" CMD config CXX' renvoie un statut 1"
See https://github.com/stan-dev/rstan/wiki/Installing-RStan-on-Windows#toolchain
Warning message:
"l'exécution de la commande 'C:/Users/JoanLlobera/.conda/envs/rl-sm-vr/lib/R/bin/x64/R CMD SHLIB file36442b921132.cpp 2> file36442b921132.cpp.err.txt' renvoie un statut 1"

Warning message:
l'exécution de la commande 'make -f "C:/Users/JOANLL~1/CONDA~1/envs/rl-sm-vr/lib/R/etc/x64/Makeconf" -f "C:/Users/JOANLL~1/CONDA~1/envs/rl-sm-vr/lib/R/share/make/winshlib.mk" SHLIB_LDFLAGS='$(SHLIB_CXXLDFLAGS)' SHLIB_LD='$(SHLIB_CXXLD)' SHLIB="file36442b921132.dll" WIN=64 TCLBIN=64 OBJECTS="file36442b921132.o"' renvoie un statut 127 

ERROR(s) during compilation: source code errors or compiler configuration errors!

Program source:
  1: 
  2: // includes from the plugin
  3: 
  4: 
  5: // user includes
  6: #define STAN__SERVICES__COMMAND_HPP// Code generated by Stan version 2.17.0
  7: 
  8: #include <stan/model/model_header.hpp>
  9: 
 10: namespace model3644154e4ab2_cb54dc7f29d639478820c0042703c2f2_namespace {
 11: 
 12: using std::istream;
 13: using std::string;
 14: using std::stringstream;
 15: using std::vector;
 16: using stan::io::dump;
 17: using stan::math::lgamma;
 18: using stan::model::prob_grad;
 19: using namespace stan::math;
 20: 
 21: typedef Eigen

ERROR: Error in compileCode(f, code, language = language, verbose = verbose): Compilation ERROR, function(s)/method(s) not created! Warning message:
l'exécution de la commande 'make -f "C:/Users/JOANLL~1/CONDA~1/envs/rl-sm-vr/lib/R/etc/x64/Makeconf" -f "C:/Users/JOANLL~1/CONDA~1/envs/rl-sm-vr/lib/R/share/make/winshlib.mk" SHLIB_LDFLAGS='$(SHLIB_CXXLDFLAGS)' SHLIB_LD='$(SHLIB_CXXLD)' SHLIB="file36442b921132.dll" WIN=64 TCLBIN=64 OBJECTS="file36442b921132.o"' renvoie un statut 127 


In [ ]:
print(fit, pars=c("a0","b0","a1","b1"), probs=c(.025,.975))

In [ ]:
e <- rstan::extract(fit)

In [ ]:
names(e)

In [ ]:
probsGT0 <- function(param){
  #finds probs of parameter values > 0
  options(digits=3)
  m <- dim(param)
  N <- m[1] #size of the sample
  k <- m[2] #number of parameters
  
  prob <- vector()
  for(i in 1:k){
    p <-sum(param[,i]>0)/N
    prob <- c(prob,p)
    print(p)
  }
  return(prob)
}


In [ ]:
#find the probabilities of the parameters being positive

mean(e$a0 > 0)
probsGT0(e$b0)

mean(e$a1 > 0)
probsGT0(e$b1)

mean(e$b0[,5] < e$b0[,1])
mean(e$b0[,5] < e$b0[,2])
mean(e$b0[,5] < e$b0[,3])
mean(e$b0[,5] < e$b0[,4])

mean(e$b0[,3] > e$b0[,1])
mean(e$b0[,3] > e$b0[,2])
mean(e$b0[,3] > e$b0[,4])
mean(e$b0[,3] > e$b0[,5])

In [ ]:
loo_summary <- function(fit, name) {

  log_lik <- extract_log_lik(fit, merge_chains = FALSE,parameter_name=name)
  r_eff <- relative_eff(exp(log_lik))
  loo <- loo(log_lik, r_eff = r_eff, cores = 2)
  return(loo)
}

library(loo)

In [ ]:
loo0 <- loo_summary(fit,"log_lik_accept0")
loo1 <- loo_summary(fit,"log_lik_accept1")

In [ ]:
loo0

In [ ]:
loo1
#showing that there is one data point with a bad diagnostic

In [ ]:
extract_until_minus1 <- function(row) {
  return(row[seq_len(which(row == -1)[1] - 1)])
}

In [ ]:
#turn into a list
configList <- apply(config, MARGIN=1, FUN=extract_until_minus1)

In [ ]:
#to get a row of the list use
configList[[1]]
length(configList[[1]])
#to get the last element
configList[[1]][length(configList[[1]])]
tail(configList[[1]],n=1)

In [ ]:
#transform each element to binary
configBinary <- lapply(configList, function(sublist) {
  lapply(sublist, intToBinary)
})

In [ ]:
#print out by row
for(i in 1:m){
  for(j in configBinary[[i]]){
    cat(j,' ')
  }
  cat("\n")
}

In [ ]:
#this gets the last configurations for each person
last <- vector()
for(i in 1:m){
  last[i] <- tail(configList[[i]],n=1)
}
table(last)
tab_last <- table(last)

In [ ]:
# Convert the table to a 2x8 matrix
matrix_last <- matrix(c(as.numeric(names(tab_last)), as.numeric(tab_last)), nrow = 2, byrow = TRUE)
for(i in matrix_last[1,]){
  print(intToBinary(i))
}
for(i in matrix_last[1,]){
  print(i)
}
for(i in matrix_last[2,]){
  print(i)
}


In [ ]:
#find the occurences of each factor in the last configurations
#vision, parallax, sound, resolution, colour

numoccurences <- vector(mode="integer",length=5)
for(f in 1:5){#each factor
  for(i in 1:m){#each person
    if(check_bit(last[i],f)) {
      numoccurences[f] <- numoccurences[f] + 1
    }
  }
}

In [ ]:
numoccurences

In [ ]:
#Markov Chain analysis
#P is the counts of the transitions matrix
#note that for some configurations never visited all entries are 0
for(i in 1:32){
  s <- sum(P[i,])
  if(s > 0){
    P[i,] <- P[i,]/s
  }
}

In [ ]:
eliminate_unreachable_states <- function(P) {
  # Find rows with all zeros (no transitions from these states)
  no_outgoing <- apply(P, 1, function(row) all(row == 0))
  
  # Find columns with all zeros (no transitions to these states)
  no_incoming <- apply(P, 2, function(col) all(col == 0))
  
  # Identify states to be eliminated
  to_eliminate <- no_outgoing & no_incoming
  
  # Eliminate states with no incoming or outgoing transitions
  P_new <- P[!to_eliminate, !to_eliminate]
  
  ind <- seq(1,32,1) #in order to idenfity the states to be eliminated
  
  # Return the modified transition matrix
  return(list(P_new = P_new, to_eliminate = ind[to_eliminate]))
}


In [ ]:
reducedMatrix <- eliminate_unreachable_states(P)
#the new matrix
Pnew <- reducedMatrix$P_new
eliminatedStates <- reducedMatrix$to_eliminate

In [ ]:
eliminatedStates

In [ ]:
#remember that the eliminatedStates refer to indices in the P matrix, not to the list of configurations
for(i in eliminatedStates) print(intToBinary(i-1))
for(i in eliminatedStates) print(i-1)


In [ ]:
#Now I need a function that maps the new state numbers to the old
mapping_vector <- setdiff(1:nrow(P), eliminatedStates)
mapToOriginal <- function(newState) {
  if (newState < 1 || newState > length(mapping_vector)) {
    stop("Invalid new state number")
  }
  return(mapping_vector[newState])
}

In [ ]:
#The equilibrium vector is the eigenvector associated with the eigenvalue 1, normalized so that its elements sum to 1.
en <- eigen(t(Pnew))

In [ ]:
index <- which.min(abs(en$values - 1))
steady_state <- Re(en$vectors[,index])
pi <- steady_state / sum(steady_state)


In [ ]:
for(i in pi) print(round(i,3))

In [ ]:
configs <- setConfigurations()

In [ ]:
ind27 <- seq(1,27)
v <- apply(as.array(ind27),1,mapToOriginal)

In [ ]:
z <- configs[v]

In [ ]:
# Get the ordering of 'pi' from greatest to least
order_indices <- order(pi, decreasing = TRUE)

# Apply this ordering to both 'pi' and 'v'
sorted_pi <- pi[order_indices]
sorted_v <- v[order_indices]

# Now 'sorted_pi' and 'sorted_v' have the same ordering


In [ ]:
#this gives the output that corresponds to Table 7
for(i in configs[sorted_v]) print(i)